In [1]:
from torchvision import models
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Recreate the same model structure
resnet18 = models.resnet18(weights=None)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 7)

# ✅ Load trained weights
resnet18.load_state_dict(torch.load("trained_models/resnet18_finetuned.pth", map_location=device))

resnet18 = resnet18.to(device)
resnet18.eval()  # very important for inference mode
print("✅ Model loaded successfully and ready for prediction!")


✅ Model loaded successfully and ready for prediction!


In [3]:
# Cell 1: imports & config
import os, time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import seaborn as sns

# Config
DATA_ROOT = "../fer2013"    # change if your extracted folder is elsewhere
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
TEST_DIR  = os.path.join(DATA_ROOT, "test")

NUM_EPOCHS = 8              # start small; increase if you have time
BATCH_SIZE = 64
LR = 1e-4
NUM_WORKERS = 4             # set 0 on Windows if you get issues
SEED = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
np.random.seed(SEED)

print("Device:", device)


Device: cuda


In [8]:
# ---- Phase 1: Model 2 (ResNet18 Transfer Learning) ----

# New transforms (ResNet expects 3 channels, 224x224)
transform_train_resnet = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_test_resnet = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Use distinct variable names to avoid conflict with baseline CNN
train_dataset_resnet = datasets.ImageFolder(TRAIN_DIR, transform=transform_train_resnet)
test_dataset_resnet  = datasets.ImageFolder(TEST_DIR, transform=transform_test_resnet)

train_loader_resnet = DataLoader(train_dataset_resnet, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader_resnet  = DataLoader(test_dataset_resnet, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f"Train samples (ResNet): {len(train_dataset_resnet)}")
print(f"Test samples (ResNet): {len(test_dataset_resnet)}")
print(f"Classes: {train_dataset_resnet.classes}")


Train samples (ResNet): 28709
Test samples (ResNet): 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
import torch

# Ensure evaluation mode and no gradient tracking
resnet18.eval()
correct = 0
total = 0

with torch.no_grad():  # no need to compute gradients during evaluation
    for images, labels in test_loader_resnet:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"✅ Test Accuracy of loaded model: {accuracy:.2f}%")


✅ Test Accuracy of loaded model: 65.55%
